In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
!ls -l

In [0]:
!git clone https://github.com/tensorflow/models

In [0]:
import os,sys
sys.path.append("/content/models/research/slim")

In [0]:
from datasets import dataset_utils
import tensorflow as tf

url = "http://download.tensorflow.org/models/resnet_v1_50_2016_08_28.tar.gz"

# Specify where you want to download the model to
checkpoints_dir = '/content'

if not tf.gfile.Exists(checkpoints_dir):
    tf.gfile.MakeDirs(checkpoints_dir)

dataset_utils.download_and_uncompress_tarball(url, checkpoints_dir)

In [0]:
!rm -rf resnet_v1_50_2016_08_28.tar.gz

In [0]:
!mkdir checkpoints

In [10]:
# download training and validation data files
validation_file_id = '1sepSrVG4_mnnmjlB-SAPorN4ksUN8jsq' 

downloaded = drive.CreateFile({'id': validation_file_id})

print(downloaded["title"], downloaded["mimeType"])

downloaded.GetContentFile(downloaded['title'])

melanoma_val.tfrecords application/octet-stream


In [11]:
training_file_id = '1inW8VfAfSYykAGqyarpigA37rUBX4C_w'

downloaded = drive.CreateFile({'id': training_file_id})

print(downloaded["title"], downloaded["mimeType"])

downloaded.GetContentFile(downloaded['title'])

melanoma_train.tfrecords application/octet-stream


In [0]:
import datetime
import numpy as np
import os
import time

def parser(record):
  """Parses input records and returns batch_size samples
  """
  keys_to_features = {
      "image": tf.FixedLenFeature((), tf.string, default_value=""),
      "label": tf.FixedLenFeature((), tf.string, default_value="")
  }
  parsed = tf.parse_single_example(record, keys_to_features)
  
  decoded_image = tf.image.decode_png(parsed["image"], channels=3)
  
  decoded_label = tf.image.decode_png(parsed["label"], channels=1) 
  
  decoded_image = tf.to_float(decoded_image)
  decoded_label = tf.image.convert_image_dtype(decoded_label, tf.float32)
  decoded_label = tf.to_int32(decoded_label)
  
  decoded_label = tf.squeeze(input=decoded_label, axis=[2]) # drop the last axis
  
  mean = tf.constant([182., 149., 135.],
                     dtype=tf.float32, shape=[1, 1, 3], name='img_mean')
  
  im_centered = decoded_image - mean
  
  return (im_centered, decoded_label)

#return {"image": im_centered,
#       "label": decoded_label}

          
def my_input_fn(filename, batch_size, epochs):
  dataset = tf.data.TFRecordDataset(
      [filename]
  ).map(
      parser
  ).shuffle(
      buffer_size=1000
  ).batch(
      batch_size
  ).repeat(
      epochs
  )
  return dataset.make_one_shot_iterator().get_next()
      
def get_deconv_filter(f_shape):
    width = f_shape[0]
    heigh = f_shape[0]
    f = np.ceil(width/2.0)
    c = (2 * f - 1 - f % 2) / (2.0 * f)
    bilinear = np.zeros([f_shape[0], f_shape[1]])
    for x in range(width):
        for y in range(heigh):
            value = (1 - abs(x / f - c)) * (1 - abs(y / f - c))
            bilinear[x, y] = value
    weights = np.zeros(f_shape)
    for i in range(f_shape[2]):
        weights[:, :, i, i] = bilinear

    init = tf.constant_initializer(value=weights,
                                   dtype=tf.float32)
    var = tf.get_variable(name="up_filter", initializer=init,
                          shape=weights.shape)
    return var

def upscore_layer(x, shape, num_classes, name, ksize, stride):
    strides = [1, stride, stride, 1]
    with tf.variable_scope(name):
        in_features = x.get_shape()[3].value
        if shape is None:
            in_shape = tf.shape(x)
            h = ((in_shape[1] - 1) * stride) + 1
            w = ((in_shape[2] - 1) * stride) + 1
            new_shape = [in_shape[0], h, w, num_classes]
        else:
            new_shape = [shape[0], shape[1], shape[2], num_classes]
        output_shape = tf.stack(new_shape)
        f_shape = [ksize, ksize, num_classes, in_features]
        num_input = ksize * ksize * in_features / stride
        stddev = (2 / num_input)**0.5
        weights = get_deconv_filter(f_shape)
        deconv = tf.nn.conv2d_transpose(x, weights, output_shape, strides = strides, padding='SAME')
        return deconv

def score_layer(x, name, num_classes, stddev = 0.001): 
    with tf.variable_scope(name) as scope:
        # get number of input channels
        in_features = x.get_shape()[3].value
        shape = [1, 1, in_features, num_classes]
        w_decay = 5e-4
        init = tf.truncated_normal_initializer(stddev = stddev)
        weights = tf.get_variable("weights", shape = shape, initializer = init)
        collection_name = tf.GraphKeys.REGULARIZATION_LOSSES

        if not tf.get_variable_scope().reuse:
            weight_decay = tf.multiply(tf.nn.l2_loss(weights), w_decay, name='weight_loss')
            tf.add_to_collection(collection_name, weight_decay)

        conv = tf.nn.conv2d(x, weights, [1, 1, 1, 1], padding='SAME')
        # Apply bias
        initializer = tf.constant_initializer(0.0)
        conv_biases = tf.get_variable(name='biases', shape=[num_classes],initializer=initializer)

        bias = tf.nn.bias_add(conv, conv_biases)

        return bias

In [0]:
batch_size=1
epochs=10
initial_learning_rate=1e-3
learning_rate_decay_factor=0.96
num_classes=2

In [0]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import resnet_v1

def model_fn(features, labels, mode, params):
    
    
    with slim.arg_scope(resnet_v1.resnet_arg_scope()):
      net, end_points = resnet_v1.resnet_v1_50(features,
                                               global_pool=False, 
                                               output_stride=32)
      
      
      scale5 = net
      scale4 = end_points['resnet_v1_50/block3/unit_5/bottleneck_v1']
      scale3 = end_points['resnet_v1_50/block2/unit_3/bottleneck_v1']
      scale2 = end_points['resnet_v1_50/block1/unit_2/bottleneck_v1']
      input_x = features
            
      # stride=32
      #resnet_v1_50/block3/unit_5/bottleneck_v1 (1, 15, 15, 1024)
      #net.shape: (1, 8, 8, 2048)  
      #resnet_v1_50/block2/unit_3/bottleneck_v1 (1, 29, 29, 512)
      #resnet_v1_50/block1/unit_2/bottleneck_v1 (1, 57, 57, 256)
      #resnet_v1_50/conv1 (1, 113, 113, 64)
        
      
    with tf.variable_scope('scale_fcn'):
        upscore2 = upscore_layer(scale5, shape = tf.shape(scale4), num_classes = num_classes, name = "upscore2", ksize = 4, stride = 2) 
        score_scale4 = score_layer(scale4, "score_scale4", num_classes = num_classes)
        fuse_scale4 = tf.add(upscore2, score_scale4)

        upscore4 = upscore_layer(fuse_scale4, shape = tf.shape(scale3), num_classes = num_classes, name = "upscore4", ksize = 4, stride = 2) 
        score_scale3 = score_layer(scale3, "score_scale3", num_classes = num_classes)
        fuse_scale3 = tf.add(upscore4, score_scale3)

        upscore8 = upscore_layer(fuse_scale3, shape = tf.shape(scale2), num_classes = num_classes, name = "upscore8", ksize = 4, stride = 2) 
        #score_scale2 = score_layer(scale2, "score_scale2", num_classes = num_classes)
        #fuse_scale2 = tf.add(upscore8, score_scale2)
        #upscore32 = upscore_layer(fuse_scale2, shape = tf.shape(input_x), num_classes = num_classes, name = "upscore32", ksize = 8, stride = 4)
        
        upscore32 = upscore_layer(upscore8, shape = tf.shape(input_x), num_classes = num_classes, name = "upscore32", ksize = 8, stride = 4)
        
        pred_up = tf.argmax(upscore32, dimension = 3)
        
        pred = tf.expand_dims(pred_up, dim = 3, name='pred')

        
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'probabilities': pred,
            'logits': pred
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    
    exclude = ['resnet_v1_50/logits', 'scale_fcn/score_scale2/biases', 
               'scale_fcn/score_scale2/weights', 'scale_fcn/score_scale3/biases',
              'scale_fcn/score_scale3/weights', 'scale_fcn/score_scale4/biases',
              'scale_fcn/score_scale4/weights', 'scale_fcn/upscore2/up_filter',
              'scale_fcn/upscore4/up_filter', 'scale_fcn/upscore8/up_filter',
              'scale_fcn/upscore32/up_filter']
    variables_to_restore = slim.get_variables_to_restore(exclude=exclude)
    tf.train.init_from_checkpoint(params['ckpt_path'],
                                  {v.name.split(':')[0]: v for v in variables_to_restore})
    
    
    # Using tf.losses, any loss is added to the tf.GraphKeys.LOSSES collection
    # We can then call the total loss easily    
    tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=upscore32)
    loss = tf.losses.get_total_loss()
    
    # Compute evaluation metrics.

    #accuracy = tf.metrics.mean_iou(labels=labels,
     #                              num_classes=2,
      #                             predictions=pred_up,
       #                            name="acc_op")
      
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)
    
    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN
    
    global_step = tf.train.get_or_create_global_step()
    
    # Variables that affect learning rate
    decay_steps=10000

    lr = tf.train.exponential_decay(initial_learning_rate,
                                    global_step,
                                    decay_steps,
                                    learning_rate_decay_factor,
                                    staircase=True)
    
    train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss, 
                                                              global_step=global_step) 

    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [0]:
    with tf.Session() as session:
        
        tf.logging.set_verbosity(tf.logging.INFO)
        
#         tensors_to_log = {'accuracy' :accuracy}
        
#         logging_tensor_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)       
        
        run_config = tf.estimator.RunConfig(save_checkpoints_steps=1000)
        
        estimator = tf.estimator.Estimator(
            model_fn=model_fn,
            model_dir='./chekpoints',
            params={
                'ckpt_path': './resnet_v1_50.ckpt'
            },
            config=run_config
        )
        
        max_steps = 10000
        
        print({'max_steps': max_steps})
        
        train_spec = tf.estimator.TrainSpec(input_fn=lambda: my_input_fn('melanoma_train.tfrecords', batch_size, epochs), max_steps=max_steps) 
        eval_spec = tf.estimator.EvalSpec(input_fn=lambda: my_input_fn('melanoma_val.tfrecords', 1, 1), start_delay_secs=5, 
                                          steps=100,
                                         throttle_secs=10)
        
        tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
            
#         # Train the Model
#         classifier.train(input_fn=lambda:train_input_fn())
        
#         classifier.evaluate(input_fn=lambda:val_input_fn())

In [0]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import resnet_v1


def parser(record):
  """Parses input records and returns batch_size samples
  """
  
  keys_to_features = {
      "image": tf.FixedLenFeature((), tf.string, default_value=""),
      "label": tf.FixedLenFeature((), tf.string, default_value="")
  }
  
  parsed = tf.parse_single_example(record, keys_to_features)
  
  decoded_image = tf.image.decode_png(parsed["image"], channels=3)
  #decoded_image = tf.image.resize_images(decoded_image, [224, 224])
  
  decoded_label = tf.image.decode_png(parsed["label"], channels=1)   
  #decoded_label = tf.image.resize_images(decoded_label, [224, 224])
  
  decoded_image = tf.to_float(decoded_image)
  decoded_label = tf.image.convert_image_dtype(decoded_label, tf.float32)
  decoded_label = tf.to_int32(decoded_label)
  
  decoded_label = tf.squeeze(input=decoded_label, axis=[2]) # drop the last axis
  
  mean = tf.constant([182., 149., 135.],
                     dtype=tf.float32, shape=[1, 1, 3], name='img_mean')
  
  im_centered = decoded_image - mean
  return (im_centered, decoded_label)
        
        #return {"image": im_centered,
         #       "label": decoded_label}
      
      
def my_input_fn(filename, batch_size, epochs):
  
  dataset = tf.data.TFRecordDataset(
      [filename]
  ).map(
      parser
  ).shuffle(
      buffer_size=1000
  ).batch(
      batch_size
  ).repeat(
      epochs
  )
  
  return dataset.make_one_shot_iterator().get_next() 
          
          
def get_deconv_filter(f_shape):
    width = f_shape[0]
    heigh = f_shape[0]
    f = np.ceil(width/2.0)
    c = (2 * f - 1 - f % 2) / (2.0 * f)
    bilinear = np.zeros([f_shape[0], f_shape[1]])
    for x in range(width):
        for y in range(heigh):
            value = (1 - abs(x / f - c)) * (1 - abs(y / f - c))
            bilinear[x, y] = value
    weights = np.zeros(f_shape)
    for i in range(f_shape[2]):
        weights[:, :, i, i] = bilinear

    init = tf.constant_initializer(value=weights,
                                   dtype=tf.float32)
    var = tf.get_variable(name="up_filter", initializer=init,
                          shape=weights.shape)
    return var

def upscore_layer(x, shape, num_classes, name, ksize, stride):
    strides = [1, stride, stride, 1]
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        in_features = x.get_shape()[3].value
        if shape is None:
            in_shape = tf.shape(x)
            h = ((in_shape[1] - 1) * stride) + 1
            w = ((in_shape[2] - 1) * stride) + 1
            new_shape = [in_shape[0], h, w, num_classes]
        else:
            new_shape = [shape[0], shape[1], shape[2], num_classes]
        output_shape = tf.stack(new_shape)
        f_shape = [ksize, ksize, num_classes, in_features]
        num_input = ksize * ksize * in_features / stride
        stddev = (2 / num_input)**0.5
        weights = get_deconv_filter(f_shape)
        deconv = tf.nn.conv2d_transpose(x, weights, output_shape, strides = strides, padding='SAME')
        return deconv

def score_layer(x, name, num_classes, stddev = 0.001): 
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE) as scope:
        # get number of input channels
        in_features = x.get_shape()[3].value
        shape = [1, 1, in_features, num_classes]
        w_decay = 5e-4
        init = tf.truncated_normal_initializer(stddev = stddev)
        weights = tf.get_variable("weights", shape = shape, initializer = init)
        collection_name = tf.GraphKeys.REGULARIZATION_LOSSES

        if not tf.get_variable_scope().reuse:
            weight_decay = tf.multiply(tf.nn.l2_loss(weights), w_decay, name='weight_loss')
            tf.add_to_collection(collection_name, weight_decay)

        conv = tf.nn.conv2d(x, weights, [1, 1, 1, 1], padding='SAME')
        # Apply bias
        initializer = tf.constant_initializer(0.0)
        conv_biases = tf.get_variable(name='biases', shape=[num_classes],initializer=initializer)

        bias = tf.nn.bias_add(conv, conv_biases)

        return bias     
      

def inference(features, labels, params):
  
  num_classes=params['num_classes']
  
  with slim.arg_scope(resnet_v1.resnet_arg_scope()):
    net, end_points = resnet_v1.resnet_v1_50(features,
                                             global_pool=False, 
                                             output_stride=32)
    
    scale5 = net
    scale4 = end_points['resnet_v1_50/block3/unit_5/bottleneck_v1']
    scale3 = end_points['resnet_v1_50/block2/unit_3/bottleneck_v1']
    scale2 = end_points['resnet_v1_50/block1/unit_2/bottleneck_v1']
    input_x = features
            
      # stride=32
      #resnet_v1_50/block3/unit_5/bottleneck_v1 (1, 15, 15, 1024)
      #net.shape: (1, 8, 8, 2048)  
      #resnet_v1_50/block2/unit_3/bottleneck_v1 (1, 29, 29, 512)
      #resnet_v1_50/block1/unit_2/bottleneck_v1 (1, 57, 57, 256)
      #resnet_v1_50/conv1 (1, 113, 113, 64)
      
  with tf.variable_scope('scale_fcn'):
    upscore2 = upscore_layer(scale5, shape = tf.shape(scale4), num_classes = num_classes, name = "upscore2", ksize = 4, stride = 2) 
    score_scale4 = score_layer(scale4, "score_scale4", num_classes = num_classes)
    fuse_scale4 = tf.add(upscore2, score_scale4)
      
    upscore4 = upscore_layer(fuse_scale4, shape = tf.shape(scale3), num_classes = num_classes, name = "upscore4", ksize = 4, stride = 2) 
    score_scale3 = score_layer(scale3, "score_scale3", num_classes = num_classes)
    fuse_scale3 = tf.add(upscore4, score_scale3)
      
    upscore8 = upscore_layer(fuse_scale3, shape = tf.shape(scale2), num_classes = num_classes, name = "upscore8", ksize = 4, stride = 2) 
    #score_scale2 = score_layer(scale2, "score_scale2", num_classes = num_classes)
    #fuse_scale2 = tf.add(upscore8, score_scale2)
    #upscore32 = upscore_layer(fuse_scale2, shape = tf.shape(input_x), num_classes = num_classes, name = "upscore32", ksize = 8, stride = 4)
      
    upscore32 = upscore_layer(upscore8, shape = tf.shape(input_x), num_classes = num_classes, name = "upscore32", ksize = 8, stride = 4)
    pred_up = tf.argmax(upscore32, dimension = 3)
    pred = tf.expand_dims(pred_up, dim = 3, name='pred')     
      
  # Using tf.losses, any loss is added to the tf.GraphKeys.LOSSES collection
  # We can then call the total loss easily    
  tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=upscore32)
  loss = tf.losses.get_total_loss()
      
  # Compute evaluation metrics.
  accuracy = tf.metrics.mean_iou(labels=labels,
                                     num_classes=2,
                                     predictions=pred_up,
                                     name="acc_op")
      
  tf.summary.scalar('accuracy', accuracy[0])
  
  global_step = tf.train.get_or_create_global_step()     
      
  # set initial learning rate and 
  # scale it by 0.1 in every 3000 steps
  boundaries = [i*params['decay_steps'] for i in range(1, int(np.ceil(params['max_steps']/params['decay_steps']))) ]
  boundaries[-1] = params['max_steps']
  initial = [params['initial_learning_rate']]
  values = initial + [params['initial_learning_rate']*(0.1**i) for i in range(1,len(boundaries)+1)]
  lr = tf.train.piecewise_constant(global_step, boundaries, values)


  # Variables that affect learning rate
  var_resnet_batchnorm = [var for var in tf.trainable_variables() if ('conv1/BatchNorm' in var.name or 'conv2/BatchNorm' in var.name or 'conv3/BatchNorm' in var.name)] 
  
  var_upscale = [var for var in tf.trainable_variables() if 'up_filter' in var.name]
  
  var_rest = [var for var in tf.trainable_variables() if var.name not in var_resnet_batchnorm+var_upscale]
  
  opt1 = tf.train.GradientDescentOptimizer(0)
  opt2 = tf.train.GradientDescentOptimizer(lr*0.1)
  opt3 = tf.train.GradientDescentOptimizer(lr)
  
  grads = tf.gradients(loss, var_resnet_batchnorm + var_upscale + var_rest)
  
  grads1 = grads[:len(var_resnet_batchnorm)]
  grads2 = grads[len(var_resnet_batchnorm):len(var_resnet_batchnorm)+len(var_upscale)]
  grads3 = grads[len(var_resnet_batchnorm)+len(var_upscale):]
  
  train_op1 = opt1.apply_gradients(zip(grads1, var_resnet_batchnorm), global_step=global_step)
  train_op2 = opt2.apply_gradients(zip(grads2, var_upscale), global_step=global_step)
  train_op3 = opt3.apply_gradients(zip(grads3, var_rest), global_step=global_step)
  
  train_op = tf.group(train_op1, train_op2, train_op3)

  #decay_steps=10000
  #lr = tf.train.exponential_decay(params['initial_learning_rate'],
  #                               global_step,
  #                              decay_steps,
  #                             params['learning_rate_decay_factor'],
  #                            staircase=True)
  
  #train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss, global_step=global_step) 
      
  return loss, accuracy[0], train_op
      

filename = 'melanoma_train.tfrecords'
batch_size=1
epochs=10
save_model_path='./checkpoints'
restore_ckpt_path='./resnet_v1_50.ckpt'
initial_learning_rate=1e-3
num_classes=2
params = {'initial_learning_rate' : initial_learning_rate,
          'num_classes' : num_classes,
          'decay_steps' : 3000,
          'max_steps' : 100000}
  
  
features_op, labels_op = my_input_fn(filename, batch_size, epochs) 
ops = inference(features_op, labels_op, params)
  
  
with tf.Session() as sess:
  
  exclude_restore = [var.name for var in tf.global_variables() if ('logits' in var.name or 'scale_fcn' in var.name) ] 
  
  
  sess.run(tf.global_variables_initializer())
  sess.run(tf.local_variables_initializer())
  
  variables_to_restore = slim.get_variables_to_restore(exclude=exclude_restore)
  tf.train.init_from_checkpoint(restore_ckpt_path,
                                {v.name.split(':')[0]: v for v in variables_to_restore})
  
  saver = tf.train.Saver()
  
  train_step = 0
  print_every = 100
  while True:
    try:
      
      loss, acc, _ = sess.run(ops)
      
      train_step += 1
      
      print('training loss: {},  step: {}'.format(loss, train_step))
      
      if train_step % print_every == 0:
        print('training mIOU: {},  step: {}'.format(acc, train_step))
      
      
    except tf.errors.OutOfRangeError:
      
      print('Training finished. Saving the resulting model to {}'.format(save_model_path))
      save_path = saver.save(sess, args.save_path)
      
      break  
    